# Proyecto Integrador 01 - Steam 

<br>

## Se realiza proceso de ETL sobre dataset user_reviews

##### Importamos las librerías

In [1]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Cebol\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


<br>
<br>

Se extraen los datos del dataset provisto

In [2]:
df_r = pd.read_csv(r'C:\Users\Cebol\OneDrive\Escritorio\Datasets\Datasets\australian_user_reviews.csv')

df_r.head()

,Unnamed: 0,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
0,0,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,0,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,0,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,1,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,1,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


<br>
<br>

Se observan la cantidad de nulos y los tipos de datos

In [3]:
dicc = {"Campo": [], "Tipo_de_dato": [], "Nulos": []}

for columna in df_r.columns:
    dicc["Campo"].append(columna)
    dicc["Tipo_de_dato"].append(df_r[columna].apply(type).unique())
    dicc["Nulos"].append(df_r[columna].isnull().sum())

df_info = pd.DataFrame(dicc)
df_info

,Campo,Tipo_de_dato,Nulos
0,Unnamed: 0,[<class 'int'>],0
1,funny,"[<class 'float'>, <class 'str'>]",51182
2,posted,"[<class 'str'>, <class 'float'>]",28
3,last_edited,"[<class 'float'>, <class 'str'>]",53193
4,item_id,[<class 'float'>],28
5,helpful,"[<class 'str'>, <class 'float'>]",28
6,recommend,"[<class 'bool'>, <class 'float'>]",28
7,review,"[<class 'str'>, <class 'float'>]",58
8,user_id,[<class 'str'>],0
9,user_url,[<class 'str'>],0


<br>
<br>

Se verifican los duplicados por user_id, item_id y review, para asegurarnos que realmente sea una review duplicada, y no un mismo usuario publicando en diferentes juegos.

In [4]:
filas_duplicadas = df_r[df_r.duplicated(subset=["user_id", "item_id", "review", "reviews"], keep=False)]
filas_duplicadas

,Unnamed: 0,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
31,9,NaN,Posted June 16.,NaN,252950.0,0 of 1 people (0%) found this review helpful,True,love it,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"{'funny': '', 'posted': 'Posted June 16.', 'la..."
115,50,NaN,"Posted December 23, 2015.",NaN,298630.0,0 of 1 people (0%) found this review helpful,True,Its so bosss,Rivtex,http://steamcommunity.com/id/Rivtex,"{'funny': '', 'posted': 'Posted December 23, 2..."
280,119,NaN,"Posted August 29, 2015.",NaN,346110.0,0 of 1 people (0%) found this review helpful,True,"It's a good game, mechanics seem solid and the...",DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"{'funny': '', 'posted': 'Posted August 29, 201..."
281,119,NaN,"Posted October 5, 2015.","Last edited October 5, 2015.",380770.0,0 of 5 people (0%) found this review helpful,False,"Well, I don't have much to say about the game....",DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"{'funny': '', 'posted': 'Posted October 5, 201..."
282,119,NaN,Posted June 29.,NaN,454890.0,No ratings yet,True,"It's a pretty good game, not bad for the 49 ce...",DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"{'funny': '', 'posted': 'Posted June 29.', 'la..."
...,...,...,...,...,...,...,...,...,...,...,...
44456,18234,NaN,Posted July 3.,NaN,422400.0,No ratings yet,True,Muy entretenido y una coleccion de armas prome...,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"{'funny': '', 'posted': 'Posted July 3.', 'las..."
44457,18234,NaN,Posted June 1.,NaN,218620.0,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,...",76561198092022514,http://steamcommunity.com/profiles/76561198092...,"{'funny': '', 'posted': 'Posted June 1.', 'las..."
44458,18234,NaN,"Posted August 17, 2014.",NaN,261820.0,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien...",76561198092022514,http://steamcommunity.com/profiles/76561198092...,"{'funny': '', 'posted': 'Posted August 17, 201..."
44459,18234,NaN,"Posted February 17, 2014.",NaN,224260.0,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"{'funny': '', 'posted': 'Posted February 17, 2..."


<br>
<br>

Se procede a eliminar los registros duplicados

In [5]:
df_r = df_r.drop_duplicates(subset=["user_id", "item_id", "review", "reviews"], keep='first')

<br>
<br>

También se decide eliminar las columnas 'funny' y 'last_edited' por su gran cantidad de Nulos, y porque no seran consideradas para nuestros endpoints de la API. Además, se decide eliminar los 28 nulos de item_id, que nos dejan solo 30 nulos en la columna review, pero que se decide no transformarlos, dado que pasarian a corresponder a reviews neutrales.

In [6]:
df_r = df_r.dropna(subset=["item_id"])
df_r = df_r.drop(columns=['funny', 'last_edited'], axis=1)
df_r.columns

Index(['Unnamed: 0', 'posted', 'item_id', 'helpful', 'recommend', 'review',
       'user_id', 'user_url', 'reviews'],
      dtype='object')

<br>
<br>

Se completan los valores nulos en reviews con espacios vacios, para que sean considerados comentarios neutros

In [7]:
df_r['review'] = df_r['review'].fillna('.')

<br>
<br>

Se elimina también la columna 'posted', dado que en la mayoría de los casos la fecha no contiene el año de posteo, entonces los datos no son de utilidad. Para el funcionamiento de la API se igualará el año de lanzamiento del juego con la review realizada.

In [8]:
df_r = df_r.drop(['posted'], axis=1)

<br>
<br>

Tambien se eliminan las columnas 'Unnamed:0' por desconocer que valor está encerrado en ella, y no encontrar ninguna relación; La columna 'user_url' por no aportar datos significativos al funcionamiento de la API; La columna 'reviews' por ser un diccionario del resto de las columnas que ya fueron desanidadas; y la columna 'helpful' por no aportar datos relevantes a mi dataframe.

In [9]:
df_r = df_r.drop(['Unnamed: 0', 'user_url', 'reviews', 'helpful'], axis=1)

<br>
<br>

Modifico la cantidad de datos para hacer el analisis de sentimiento, para poder renderizar

In [10]:
df_render = df_r.head(1500)

<br>
<br>

## Feature Engineering

<br>

Se procede con la creación de la columna 'sentiment_analysis' y la visualización final del dataframe

In [15]:
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  
    elif isinstance(text, str):
        sentiment = SentimentIntensityAnalyzer().polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1  # Neutral
    else:
        return 1  

# Aplicar el método .loc para evitar la advertencia "SettingWithCopyWarning"
df_render.loc[:, 'review'] = df_render['review'].str.lower()
df_render.loc[:, 'sentiment_analysis'] = df_render['review'].apply(get_sentiment_score)



<br>
<br>

El dataset quedaria conformado de la siguiente manera

In [17]:
df_render

,item_id,recommend,review,user_id,sentiment_analysis
0,1250.0,True,simple yet with great replayability. in my opi...,76561197970982479,2
1,22200.0,True,it's unique and worth a playthrough.,76561197970982479,2
2,43110.0,True,great atmosphere. the gunplay can be a bit chu...,76561197970982479,2
3,251610.0,True,i know what you think when you see this title ...,js41637,2
4,227300.0,True,for a simple (it's actually not all that simpl...,js41637,2
...,...,...,...,...,...
1498,4000.0,True,i sell ababies to people through this game 10\10,76561198074347537,1
1499,730.0,True,very good,SurfChicken,2
1500,236390.0,False,this game used to be a fun game with an even p...,76561198126142125,0
1501,730.0,False,for everyone tempted to buy cs go... don't tru...,Clouwolf,2


In [18]:
dicc = {"Campo": [], "Tipo_de_dato": [], "Nulos": []}

for columna in df_render.columns:
    dicc["Campo"].append(columna)
    dicc["Tipo_de_dato"].append(df_render[columna].apply(type).unique())
    dicc["Nulos"].append(df_render[columna].isnull().sum())

df_info = pd.DataFrame(dicc)
df_info

,Campo,Tipo_de_dato,Nulos
0,item_id,[<class 'float'>],0
1,recommend,[<class 'bool'>],0
2,review,[<class 'str'>],0
3,user_id,[<class 'str'>],0
4,sentiment_analysis,[<class 'int'>],0


<br>

#### Se carga el Dataset limpio

In [19]:
df_render.to_csv(r'C:\Users\Cebol\OneDrive\Escritorio\Datasets\Datasets\australian_user_reviews_limpio.csv', index=False, encoding='utf-8')